## Generate objects along the Trail with Overpass
* this [notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Generate%20objects%20along%20the%20Trail%20with%20Overpass.ipynb)
   * [Notebook that checks if area](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Python%20snippet%20that%20checks%20which%20of%20these%20area%20IDs%20return%20valid%20results.ipynb) returns data
   * [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/toilets_and_water_SAT_2025-07-24_0713.ipynb) Test skapa karta som visar fel på objekten med Förbättringsförslag
* issue Generate objects along the Trail see [#118 Städa OSM data kring leden](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/118) / 
   * generates geojson
* https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/117

      nwr["amenity"="toilets"](area.searchArea);
      nwr["amenity"="drinking_water"](area.searchArea);
      nwr["drinking_water"]["drinking_water"!="no"](area.searchArea);

* exempel genererad fil https://gist.github.com/salgo60/7afc1dbeaf87edb6ce7ffd14a58991e0
   * [raw](https://gist.githubusercontent.com/salgo60/7afc1dbeaf87edb6ce7ffd14a58991e0/raw/f8074bede0631c385e93949304d02bc7756c8ebc/SAT%2520toilets%2520Water.geojson)
 
---- 
#### Karta uppladdad till GITHUB
* [Stockholm_Archipelago_Trail/html/toaletter_vatten_sat_karta.html](https://salgo60.github.io/Stockholm_Archipelago_Trail/html/toaletter_vatten_sat_karta.html)

### TODO 
* missing a lot of objects returned in the Overpass query see problem with OSM ways https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/116#issuecomment-3113411094


In [1]:
import time
from datetime import datetime

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-07-24 11:38:34


In [7]:
# SAT Toilets & Drinking Water Points Extractor for uMap

import sys
import time
import requests
import geopandas as gpd
from shapely.geometry import Point
from SPARQLWrapper import SPARQLWrapper, JSON 
from IPython.display import Image, display


# --- Step 1: Get SAT islands with Overpass area IDs from Wikidata ---
endpoint_url = "https://query.wikidata.org/sparql"

query = """
PREFIX wd:   <http://www.wikidata.org/entity/>
PREFIX wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

SELECT ?item ?itemLabel ?placedOn ?placedOnLabel ?OSMareaRel ?OSMareaWay ?OSMareanumber ?overpassAreaQL
WHERE {
  ?item wdt:P361 wd:Q131318799 ;
        wdt:P31  wd:Q2143825 .

  OPTIONAL { ?item wdt:P706 ?placedOn. }

  OPTIONAL {
    ?placedOn wdt:P402 ?OSMrel .
    BIND( xsd:integer(?OSMrel) + 3600000000 AS ?OSMareaRel )
  }

  OPTIONAL {
    ?placedOn wdt:P10689 ?OSMway .
    BIND( xsd:integer(?OSMway) + 2400000000 AS ?OSMareaWay )
  }

  BIND( COALESCE(?OSMareaRel, ?OSMareaWay) AS ?OSMareanumber )
  BIND(
    IF(BOUND(?OSMareanumber),
       CONCAT("area(", STR(?OSMareanumber), ")"),
       ""
    ) AS ?overpassAreaQL
  )

  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "sv,en" .
  }
}
ORDER BY ?itemLabel
"""

# Run SPARQL query
def get_results(endpoint_url, query):
    user_agent = "SAT-WD-Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query)

# --- Step 2: Build list of Overpass areas to query ---
area_list = []
for r in results["results"]["bindings"]:
    if "overpassAreaQL" in r and r["overpassAreaQL"]["value"]:
        area_id = r["overpassAreaQL"]["value"].replace("area(", "").replace(")", "")
        island = r.get("placedOnLabel", {}).get("value", "Unknown")
        area_list.append({"area_id": area_id, "island": island})

# --- Step 3: For each island, query Overpass API for toilets & drinking water ---
features = []
for area in area_list:
    print(f"✅ Query: {area['island']} - {area['area_id']} ")    
    query = f"""
    [out:json][timeout:25];
    area({area['area_id']})->.searchArea;
    (
      nwr["amenity"="toilets"](area.searchArea);
      nwr["amenity"="drinking_water"]["drinking_water"!="no"](area.searchArea);
      nwr["drinking_water"]["drinking_water"!="no"](area.searchArea);
    );    
    out tags center;
    """
    response = requests.get("https://overpass-api.de/api/interpreter", params={"data": query})
    if response.status_code == 200:
        data = response.json()
        for el in data["elements"]:
            if "lat" in el and "lon" in el:
                geom = Point(el["lon"], el["lat"])
            elif "center" in el:
                geom = Point(el["center"]["lon"], el["center"]["lat"])
            else:
                continue
            tags = el.get("tags", {})
            tags.update({
                "osm_id": el["id"],
                "osm_type": el["type"],
                "island": area["island"]
            })
            features.append({"geometry": geom, "properties": tags})
            print(f"\t {el['id']} - {el['type']} - https://www.openstreetmap.org/{el['type']}/{el['id']}  ")    
            if 'image' in el and el['image'].startswith('http'):
                print(f"\t\t image")
                display(Image(url=el['image']))
            time.sleep(1.2)  # Overpass API rate limit

# --- Step 4: Convert to GeoDataFrame and export as GeoJSON ---
gdf = gpd.GeoDataFrame(
    [f["properties"] for f in features],
    geometry=[f["geometry"] for f in features],
    crs="EPSG:4326"
)

today_str_filename = datetime.now().strftime("%Y-%m-%d_%H%M")
output_path = f"toilets_and_water_SAT_{today_str_filename}.geojson" 
gdf.to_file(output_path, driver="GeoJSON")
print(f"✅ Exported: {output_path} with {len(gdf)} features - ")
display(gdf.head())


✅ Query: Arholma - 3606930812 
	 1848869434 - node - https://www.openstreetmap.org/node/1848869434  
	 1848869435 - node - https://www.openstreetmap.org/node/1848869435  
	 2960940412 - node - https://www.openstreetmap.org/node/2960940412  
	 5744466280 - node - https://www.openstreetmap.org/node/5744466280  
	 5745003238 - node - https://www.openstreetmap.org/node/5745003238  
	 5745003240 - node - https://www.openstreetmap.org/node/5745003240  
	 5745003241 - node - https://www.openstreetmap.org/node/5745003241  
	 12992825691 - node - https://www.openstreetmap.org/node/12992825691  
	 12992830698 - node - https://www.openstreetmap.org/node/12992830698  
	 292553972 - way - https://www.openstreetmap.org/way/292553972  
✅ Query: Finnhamn - 3614477930 
	 1401738704 - node - https://www.openstreetmap.org/node/1401738704  
	 1401739421 - node - https://www.openstreetmap.org/node/1401739421  
	 1414776609 - node - https://www.openstreetmap.org/node/1414776609  
	 2908785465 - node - https

,access,amenity,changing_table,fee,operator,toilets:disposal,unisex,wheelchair,osm_id,osm_type,...,hot_water,covered,drinking_water:legal,opening_hours,toilets:paper_supplied,highway,cold_water,warm_water,drinking_water:seasonal,geometry
0,yes,toilets,no,no,Norrtälje kommun,bucket,yes,no,1848869434,node,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (19.11926 59.8427)
1,NaN,drinking_water,NaN,NaN,NaN,NaN,NaN,NaN,1848869435,node,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (19.11888 59.84245)
2,customers,toilets,NaN,NaN,NaN,NaN,NaN,NaN,2960940412,node,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (19.12253 59.85984)
3,yes,toilets,NaN,NaN,NaN,bucket,yes,NaN,5744466280,node,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (19.11314 59.85327)
4,NaN,toilets,NaN,NaN,NaN,NaN,yes,NaN,5745003238,node,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (19.12351 59.86088)


In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2025-07-24 11:41:28
Total time elapsed: 174.03 seconds


In [4]:
area

{'area_id': '3606315443', 'island': 'Utö'}